## Section 1 Overview
In this section, we build the Fuzzy Logic system:
- Data cleaning and preprocessing
- Definition of fuzzy variables and membership functions
- Construction of the initial FIS rule base
- Implementation of the prediction function


#Section 1

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.2 MB/s eta 0:00:00


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell 1: Import Libraries

import numpy as np
import pandas as pd

# Fuzzy Logic
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Data Preprocessing & Model Selection
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Machine Learning Models
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

# Evaluation Metrics
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell 2: Load and Inspect Dataset

import pandas as pd

# Load dataset into variable `df`
df = pd.read_excel('/content/sample_data/Health_Risk_Dataset_Assignment.xlsx')

# Inspect the first few rows and column names
print("Columns:", df.columns.tolist())
print("\nFirst 5 rows:")
print(df.head())


Columns: ['PatientID', 'Age', 'Height', 'HeartRate', 'BloodPressure', 'OxygenSaturation', 'StressLevel', 'ActivityLevel', 'BMI', 'HealthRisk']

First 5 rows:
   PatientID   Age  Height  HeartRate  BloodPressure  OxygenSaturation  \
0          1  75.0    1.68       57.0          150.0              98.0   
1          2  53.0    1.59       83.0          151.0              95.0   
2          3  49.0    1.65      138.0          163.0              92.0   
3          4  40.0    1.56      139.0          157.0              92.0   
4          5  57.0    1.91       67.0            NaN              97.0   

   StressLevel ActivityLevel    BMI  HealthRisk  
0            6        Medium  21.77           1  
1            5          High  21.24           1  
2            3        Medium  28.65           1  
3            1          High  25.81           1  
4            1           Low  25.10           1  


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell 3: Check and Impute Null Values, Round to 1 Decimal

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Assuming df is already loaded
# 1. Check null values before imputation
print("Null values before imputation:")
print(df.isnull().sum(), "\n")

# 2. Identify numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# 3. Impute numeric columns with mean
imputer = SimpleImputer(strategy='mean')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# 4. Round numeric columns to one decimal place
df[numeric_cols] = df[numeric_cols].round(1)

# 5. Check null values after imputation
print("Null values after imputation:")
print(df.isnull().sum(), "\n")

# 6. Preview cleaned dataset
print("Cleaned dataset preview:")
print(df.head())


Null values before imputation:
PatientID             0
Age                 623
Height              623
HeartRate           621
BloodPressure       617
OxygenSaturation    621
StressLevel           0
ActivityLevel         0
BMI                 625
HealthRisk            0
dtype: int64 

Null values after imputation:
PatientID           0
Age                 0
Height              0
HeartRate           0
BloodPressure       0
OxygenSaturation    0
StressLevel         0
ActivityLevel       0
BMI                 0
HealthRisk          0
dtype: int64 

Cleaned dataset preview:
   PatientID   Age  Height  HeartRate  BloodPressure  OxygenSaturation  \
0        1.0  75.0     1.7       57.0          150.0              98.0   
1        2.0  53.0     1.6       83.0          151.0              95.0   
2        3.0  49.0     1.6      138.0          163.0              92.0   
3        4.0  40.0     1.6      139.0          157.0              92.0   
4        5.0  57.0     1.9       67.0          134.5  

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
df.head()

,PatientID,Age,Height,HeartRate,BloodPressure,OxygenSaturation,StressLevel,ActivityLevel,BMI,HealthRisk
0,1.0,75.0,1.7,57.0,150.0,98.0,6.0,Medium,21.8,1.0
1,2.0,53.0,1.6,83.0,151.0,95.0,5.0,High,21.2,1.0
2,3.0,49.0,1.6,138.0,163.0,92.0,3.0,Medium,28.6,1.0
3,4.0,40.0,1.6,139.0,157.0,92.0,1.0,High,25.8,1.0
4,5.0,57.0,1.9,67.0,134.5,97.0,1.0,Low,25.1,1.0


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell 4: Define Fuzzy Variables, Membership Functions, 24 Rules, and Prediction Function

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# 1. Define Antecedents (inputs) and Consequent (output)
hr = ctrl.Antecedent(np.arange(40, 181, 1), 'heart_rate')
bp = ctrl.Antecedent(np.arange(80, 201, 1), 'blood_pressure')
ox = ctrl.Antecedent(np.arange(50, 101, 1), 'oxygen_saturation')
act = ctrl.Antecedent(np.arange(0, 11, 1), 'activity_level')
st = ctrl.Antecedent(np.arange(0, 11, 1), 'stress_level')
ag = ctrl.Antecedent(np.arange(0, 101, 1), 'age')
ht = ctrl.Antecedent(np.arange(100, 221, 1), 'height')
bm = ctrl.Antecedent(np.arange(10, 51, 1), 'bmi')
risk = ctrl.Consequent(np.arange(0, 101, 1), 'health_risk')

# 2. Membership Functions
hr['low']    = fuzz.trapmf(hr.universe, [40, 40, 60, 75])
hr['normal'] = fuzz.trimf(hr.universe, [60, 75, 100])
hr['high']   = fuzz.trapmf(hr.universe, [90, 110, 180, 180])

bp['low']    = fuzz.trapmf(bp.universe, [80, 80, 90, 110])
bp['normal'] = fuzz.trimf(bp.universe, [90, 120, 140])
bp['high']   = fuzz.trapmf(bp.universe, [130, 150, 200, 200])

ox['low']    = fuzz.trapmf(ox.universe, [50, 50, 85, 90])
ox['normal'] = fuzz.trimf(ox.universe, [85, 95, 100])

act['low']      = fuzz.trimf(act.universe, [0, 0, 4])
act['moderate'] = fuzz.trimf(act.universe, [3, 5, 7])
act['high']     = fuzz.trimf(act.universe, [6, 10, 10])

st['low']    = fuzz.trimf(st.universe, [0, 0, 3])
st['medium'] = fuzz.trimf(st.universe, [2, 5, 8])
st['high']   = fuzz.trimf(st.universe, [7, 10, 10])

ag['young']  = fuzz.trapmf(ag.universe, [0, 0, 20, 30])
ag['middle'] = fuzz.trimf(ag.universe, [25, 45, 65])
ag['old']    = fuzz.trapmf(ag.universe, [60, 70, 100, 100])

ht['short'] = fuzz.trapmf(ht.universe, [100, 100, 140, 160])
ht['avg']   = fuzz.trimf(ht.universe, [150, 170, 190])
ht['tall']  = fuzz.trapmf(ht.universe, [180, 200, 220, 220])

bm['underweight'] = fuzz.trapmf(bm.universe, [10, 10, 15, 18.5])
bm['normal']      = fuzz.trimf(bm.universe, [18.5, 22, 25])
bm['overweight']  = fuzz.trimf(bm.universe, [24, 27.5, 30])
bm['obese']       = fuzz.trapmf(bm.universe, [29, 32, 50, 50])

risk['low']  = fuzz.trimf(risk.universe, [0, 0, 50])
risk['high'] = fuzz.trimf(risk.universe, [50, 100, 100])

# 3. Define 24 Fuzzy Rules
rules = [
    # High-risk rules (12)
    ctrl.Rule(hr['high']   & bp['high'],     risk['high']),
    ctrl.Rule(hr['high']   & ox['low'],      risk['high']),
    ctrl.Rule(hr['high']   & st['high'],     risk['high']),
    ctrl.Rule(bp['high']   & ox['low'],      risk['high']),
    ctrl.Rule(bp['high']   & st['high'],     risk['high']),
    ctrl.Rule(ox['low']    & st['high'],     risk['high']),
    ctrl.Rule(st['high']   & bm['obese'],    risk['high']),
    ctrl.Rule(ag['old']    & bp['high'],     risk['high']),
    ctrl.Rule(ag['old']    & hr['high'],     risk['high']),
    ctrl.Rule(ag['old']    & st['high'],     risk['high']),
    ctrl.Rule(act['low']   & bm['obese'],    risk['high']),
    ctrl.Rule(bm['obese']  & st['medium'],   risk['high']),

    # Low-risk rules (12)
    ctrl.Rule(hr['normal'] & bp['normal'] & ox['normal'], risk['low']),
    ctrl.Rule(hr['normal'] & act['high'],   risk['low']),
    ctrl.Rule(bp['normal'] & act['high'],   risk['low']),
    ctrl.Rule(ox['normal'] & act['moderate'], risk['low']),
    ctrl.Rule(st['low']    & act['high'],   risk['low']),
    ctrl.Rule(bm['normal'] & ag['young'],   risk['low']),
    ctrl.Rule(ag['young']  & ht['tall'],    risk['low']),
    ctrl.Rule(bm['normal'] & ox['normal'],  risk['low']),
    ctrl.Rule(bp['normal'] & st['low'],     risk['low']),
    ctrl.Rule(hr['low']    & ox['normal'],  risk['low']),
    ctrl.Rule(act['moderate'] & ht['avg'],  risk['low']),
    ctrl.Rule(ag['middle'] & hr['normal'],  risk['low'])
]

# 4. Build Control System and Simulation
health_ctrl = ctrl.ControlSystem(rules)
health_sim = ctrl.ControlSystemSimulation(health_ctrl)

# 5. Prediction Function
def predict_binary_risk(heart_rate, blood_pressure, oxygen_saturation,
                        activity_level, stress_level, age, height, bmi):
    """
    Returns (risk_score, risk_binary)
    """
    health_sim.input['heart_rate']        = heart_rate
    health_sim.input['blood_pressure']    = blood_pressure
    health_sim.input['oxygen_saturation'] = oxygen_saturation
    health_sim.input['activity_level']    = activity_level
    health_sim.input['stress_level']      = stress_level
    health_sim.input['age']               = age
    health_sim.input['height']            = height
    health_sim.input['bmi']               = bmi
    health_sim.compute()
    score = round(health_sim.output['health_risk'], 1)
    return score, int(score >= 50)

# Example usage:
# score, binary = predict_binary_risk(80, 120, 98, 5, 4, 45, 170, 24)
# print(f"Risk Score: {score}, Binary: {binary}")


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Example usage (replace with your own values)
example = {
    'heart_rate': 180,
    'blood_pressure': 220,
    'oxygen_saturation': 98,
    'activity_level': 5,
    'stress_level': 4,
    'age': 45,
    'height': 1.7,
    'bmi': 24
}
score, binary = predict_binary_risk(**example)
print(f"Risk Score: {score:.1f}, Binary Risk: {binary}")


Risk Score: 58.8, Binary Risk: 1


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell 7: Interactive User Input and Fuzzy Prediction

# Assumes `predict_binary_risk` is already defined in the notebook

# Prompt the user for each input value
heart_rate       = float(input("Enter Heart Rate (bpm): "))
blood_pressure   = float(input("Enter Blood Pressure (mmHg): "))
oxygen_saturation = float(input("Enter Oxygen Saturation (%): "))
activity_level   = float(input("Enter Activity Level (0 - 10): "))
stress_level     = float(input("Enter Stress Level (0 - 10): "))
age              = float(input("Enter Age (years): "))
height           = float(input("Enter Height (m): "))
bmi              = float(input("Enter BMI: "))

# Run the fuzzy prediction function
risk_score, risk_binary = predict_binary_risk(
    heart_rate, blood_pressure, oxygen_saturation,
    activity_level, stress_level, age, height, bmi
)

# Display the results
print(f"\nPredicted Health Risk Score: {risk_score:.1f} / 100")
print("Predicted Risk Category: ", "HIGH RISK" if risk_binary else "LOW RISK")


Enter Heart Rate (bpm): 88
Enter Blood Pressure (mmHg): 110
Enter Oxygen Saturation (%): 96
Enter Activity Level (0=Low, 5=Medium, 10=High): 6
Enter Stress Level (0=Low, 5=Medium, 10=High): 2
Enter Age (years): 23
Enter Height (cm): 1.8
Enter BMI: 22.6

Predicted Health Risk Score: 17.2 / 100
Predicted Risk Category:  LOW RISK


## Section 2 Overview
In this section, we train ML and DL models:
- Prepare feature matrix and target
- Train Random Forest, XGBoost, Decision Tree
- Train MLP, LSTM, GRU, RNN models
- Evaluate performance metrics


#Section 2

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-1: Random Forest on Cleaned Dataset

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Assume df is the cleaned DataFrame already in memory,
# with 'ActivityLevel' as strings and 'StressLevel' numeric 1-10.

# 1. Map ActivityLevel to numeric (Low=0, Medium=5, High=10)
df['ActivityLevel_num'] = df['ActivityLevel'].map({'Low': 0, 'Medium': 1, 'High': 2})

# 2. Define features X and target y
feature_cols = ['Age', 'Height', 'HeartRate', 'BloodPressure',
                'OxygenSaturation', 'ActivityLevel_num', 'StressLevel', 'BMI']
X = df[feature_cols]
y = df['HealthRisk'].astype(int)

# 3. Split into training and test sets (70/30 stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 5. Train Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)

# 6. Predict and evaluate
y_pred  = rf.predict(X_test_scaled)
y_proba = rf.predict_proba(X_test_scaled)[:, 1]

print("=== Random Forest Performance ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba):.3f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))


=== Random Forest Performance ===
Accuracy: 0.983
ROC AUC:   0.997

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       532
           1       0.99      0.99      0.99      3218

    accuracy                           0.98      3750
   macro avg       0.96      0.97      0.97      3750
weighted avg       0.98      0.98      0.98      3750



### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
df.head()

,PatientID,Age,Height,HeartRate,BloodPressure,OxygenSaturation,StressLevel,ActivityLevel,BMI,HealthRisk,ActivityLevel_num
0,1.0,75.0,1.7,57.0,150.0,98.0,6.0,Medium,21.8,1.0,1
1,2.0,53.0,1.6,83.0,151.0,95.0,5.0,High,21.2,1.0,2
2,3.0,49.0,1.6,138.0,163.0,92.0,3.0,Medium,28.6,1.0,1
3,4.0,40.0,1.6,139.0,157.0,92.0,1.0,High,25.8,1.0,2
4,5.0,57.0,1.9,67.0,134.5,97.0,1.0,Low,25.1,1.0,0


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-6: Interactive Random Forest Prediction

# Assumes `scaler` and `rf` (trained RandomForestClassifier) are available in the notebook

# 1. Prompt user for input values
age               = float(input("Enter Age (years): "))
height            = float(input("Enter Height (cm): "))
heart_rate        = float(input("Enter Heart Rate (bpm): "))
blood_pressure    = float(input("Enter Blood Pressure (mmHg): "))
oxygen_saturation = float(input("Enter Oxygen Saturation (%): "))
activity_level    = float(input("Enter Activity Level (0=Low, 1=Medium, 2=High): "))
stress_level      = float(input("Enter Stress Level (0=Low, 5=Medium, 10=High): "))
bmi               = float(input("Enter BMI: "))

# 2. Create feature array
import numpy as np
X_new = np.array([[age, height, heart_rate, blood_pressure,
                   oxygen_saturation, activity_level, stress_level, bmi]])

# 3. Scale the input
X_new_scaled = scaler.transform(X_new)

# 4. Predict with Random Forest
pred_label = rf.predict(X_new_scaled)[0]
pred_proba = rf.predict_proba(X_new_scaled)[0, 1]

# 5. Display the results
risk_text = "HIGH RISK" if pred_label == 1 else "LOW RISK"
print(f"\nRandom Forest Prediction: {risk_text}")
print(f"Predicted Probability of High Risk: {pred_proba:.3f}")


Enter Age (years): 23
Enter Height (cm): 1.8
Enter Heart Rate (bpm): 88
Enter Blood Pressure (mmHg): 110
Enter Oxygen Saturation (%): 96
Enter Activity Level (0=Low, 1=Medium, 2=High): 2
Enter Stress Level (0=Low, 5=Medium, 10=High): 4
Enter BMI: 22.6

Random Forest Prediction: LOW RISK
Predicted Probability of High Risk: 0.030


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-8: Train and Evaluate XGBoost Classifier (Baseline)

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Assumes `X_train_scaled`, `X_test_scaled`, `y_train`, `y_test` are defined from the Random Forest section

# Initialize and train XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred_xgb  = xgb_model.predict(X_test_scaled)
y_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]

print("=== XGBoost Performance ===")
print("Accuracy:", round(accuracy_score(y_test, y_pred_xgb), 3))
print("ROC AUC:", round(roc_auc_score(y_test, y_proba_xgb), 3))
print("\nClassification Report:\n", classification_report(y_test, y_pred_xgb))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:43:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


=== XGBoost Performance ===
Accuracy: 0.982
ROC AUC: 0.996

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.94      0.94       532
           1       0.99      0.99      0.99      3218

    accuracy                           0.98      3750
   macro avg       0.96      0.97      0.96      3750
weighted avg       0.98      0.98      0.98      3750



### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
df.head()

,PatientID,Age,Height,HeartRate,BloodPressure,OxygenSaturation,StressLevel,ActivityLevel,BMI,HealthRisk,ActivityLevel_num
0,1.0,75.0,1.7,57.0,150.0,98.0,6.0,Medium,21.8,1.0,1
1,2.0,53.0,1.6,83.0,151.0,95.0,5.0,High,21.2,1.0,2
2,3.0,49.0,1.6,138.0,163.0,92.0,3.0,Medium,28.6,1.0,1
3,4.0,40.0,1.6,139.0,157.0,92.0,1.0,High,25.8,1.0,2
4,5.0,57.0,1.9,67.0,134.5,97.0,1.0,Low,25.1,1.0,0


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-9: Interactive XGBoost Prediction

# Assumes `scaler` and `xgb_model` are already defined in the notebook

# 1. Prompt user for input values
age               = float(input("Enter Age (years): "))
height            = float(input("Enter Height (cm): "))
heart_rate        = float(input("Enter Heart Rate (bpm): "))
blood_pressure    = float(input("Enter Blood Pressure (mmHg): "))
oxygen_saturation = float(input("Enter Oxygen Saturation (%): "))
activity_level    = float(input("Enter Activity Level (0=Low, 5=Medium, 10=High): "))
stress_level      = float(input("Enter Stress Level (0=Low, 5=Medium, 10=High): "))
bmi               = float(input("Enter BMI: "))

# 2. Create feature array
import numpy as np
X_new = np.array([[age, height, heart_rate, blood_pressure,
                   oxygen_saturation, activity_level, stress_level, bmi]])

# 3. Scale the input
X_new_scaled = scaler.transform(X_new)

# 4. Predict with XGBoost
pred_label = xgb_model.predict(X_new_scaled)[0]
pred_proba = xgb_model.predict_proba(X_new_scaled)[0, 1]

# 5. Display the results
risk_text = "HIGH RISK" if pred_label == 1 else "LOW RISK"
print(f"\nXGBoost Prediction: {risk_text}")
print(f"Predicted Probability of High Risk: {pred_proba:.3f}")


Enter Age (years): 23
Enter Height (cm): 1.8
Enter Heart Rate (bpm): 88
Enter Blood Pressure (mmHg): 110
Enter Oxygen Saturation (%): 96
Enter Activity Level (0=Low, 5=Medium, 10=High): 10
Enter Stress Level (0=Low, 5=Medium, 10=High): 0
Enter BMI: 23.4

XGBoost Prediction: LOW RISK
Predicted Probability of High Risk: 0.003


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-DT: Train and Evaluate a Decision Tree Classifier

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Assume `df` is your cleaned DataFrame in memory with numeric features

# 1. Define feature matrix X and target y
# Convert 'ActivityLevel' to numeric before training
df['ActivityLevel_num'] = df['ActivityLevel'].map({'Low': 0, 'Medium': 1, 'High': 2})
feature_cols = ['Age', 'Height', 'HeartRate', 'BloodPressure',
                'OxygenSaturation', 'ActivityLevel_num', 'StressLevel', 'BMI'] # Use the new numeric column
X = df[feature_cols]
y = df['HealthRisk'].astype(int)

# 2. Split into train and test sets (70/30 stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 3. Train Decision Tree classifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# 4. Predict and evaluate
y_pred  = dt.predict(X_test)
# Use predict_proba for ROC AUC
y_proba = dt.predict_proba(X_test)[:, 1]

print("=== Decision Tree Performance ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba):.3f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

=== Decision Tree Performance ===
Accuracy: 0.978
ROC AUC:   0.952

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.92      0.92       532
           1       0.99      0.99      0.99      3218

    accuracy                           0.98      3750
   macro avg       0.96      0.95      0.95      3750
weighted avg       0.98      0.98      0.98      3750



### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-13: Interactive Linear Regression Prediction

# Assumes `scaler` and `lr` (trained LinearRegression) are available in the notebook

# 1. Prompt user for input values
age               = float(input("Enter Age (years): "))
height            = float(input("Enter Height (cm): "))
heart_rate        = float(input("Enter Heart Rate (bpm): "))
blood_pressure    = float(input("Enter Blood Pressure (mmHg): "))
oxygen_saturation = float(input("Enter Oxygen Saturation (%): "))
activity_level    = float(input("Enter Activity Level (0=Low, 5=Medium, 10=High): "))
stress_level      = float(input("Enter Stress Level (0=Low, 5=Medium, 10=High): "))
bmi               = float(input("Enter BMI: "))

# 2. Create feature array
import numpy as np
X_new = np.array([[age, height, heart_rate, blood_pressure,
                   oxygen_saturation, activity_level, stress_level, bmi]])

# 3. Scale the input
X_new_scaled = scaler.transform(X_new)

# 4. Predict with Linear Regression
pred_cont = dt.predict(X_new_scaled)[0]
pred_bin  = int(pred_cont >= 0.5)

# 5. Display the results
print(f"\nPredicted Continuous Output (Risk Probability): {pred_cont:.3f}")
print("Predicted Risk Category:", "HIGH RISK" if pred_bin else "LOW RISK")


Enter Age (years): 23
Enter Height (cm): 1.8
Enter Heart Rate (bpm): 88
Enter Blood Pressure (mmHg): 110
Enter Oxygen Saturation (%): 96
Enter Activity Level (0=Low, 5=Medium, 10=High): 10
Enter Stress Level (0=Low, 5=Medium, 10=High): 0
Enter BMI: 23.4

Predicted Continuous Output (Risk Probability): 0.000
Predicted Risk Category: LOW RISK


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-LSTM-Fix: LSTM with NaN Handling

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Cell ML-LSTM-Fix: LSTM with NaN Handling

# ... (previous code)

# 1. Prepare feature matrix X and target vector y (assumes df is cleaned)
# Convert 'ActivityLevel' to numeric before using it in LSTM
df['ActivityLevel_num'] = df['ActivityLevel'].map({'Low': 0, 'Medium': 1, 'High': 2})
X = df[['Age','Height','HeartRate','BloodPressure',
        'OxygenSaturation','ActivityLevel_num','StressLevel','BMI']].values # Use the new numeric column
y = df['HealthRisk'].astype(int).values

# ... (rest of the code)

# 2. Split into train/test (70/30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 3. Scale features
scaler_lstm = StandardScaler()
X_train_scaled = scaler_lstm.fit_transform(X_train)
X_test_scaled  = scaler_lstm.transform(X_test)

# 3a. Check for NaNs/Infs
print("NaNs in X_train_scaled:", np.isnan(X_train_scaled).sum())
print("Infs in X_train_scaled:", np.isinf(X_train_scaled).sum())

# 3b. Replace NaNs/Infs with 0.0
X_train_scaled = np.nan_to_num(X_train_scaled, nan=0.0, posinf=0.0, neginf=0.0)
X_test_scaled  = np.nan_to_num(X_test_scaled,  nan=0.0, posinf=0.0, neginf=0.0)

# 4. Reshape for LSTM: [samples, timesteps=1, features]
X_train_lstm = X_train_scaled.reshape((-1, 1, X_train_scaled.shape[1]))
X_test_lstm  = X_test_scaled.reshape((-1, 1, X_test_scaled.shape[1]))

# 5. Build LSTM Model with explicit Input
model = Sequential([
    Input(shape=(1, X_train_lstm.shape[2])),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 6. Train
history = model.fit(
    X_train_lstm, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test_lstm, y_test),
    verbose=1
)

# 7. Evaluate
y_proba_lstm = model.predict(X_test_lstm).flatten()
y_pred_lstm  = (y_proba_lstm >= 0.5).astype(int)

print("=== LSTM Performance ===")
print("Accuracy:", round(accuracy_score(y_test, y_pred_lstm), 3))

# Only compute ROC-AUC if there are no NaNs
if not np.isnan(y_proba_lstm).any():
    print("ROC AUC:", round(roc_auc_score(y_test, y_proba_lstm), 3))
else:
    print("ROC AUC: skipped (NaNs in predictions)")

print("\nClassification Report:\n", classification_report(y_test, y_pred_lstm))


NaNs in X_train_scaled: 0
Infs in X_train_scaled: 0
Epoch 1/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8023 - loss: 0.5257 - val_accuracy: 0.9051 - val_loss: 0.2275
Epoch 2/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9040 - loss: 0.2253 - val_accuracy: 0.9181 - val_loss: 0.1812
Epoch 3/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9174 - loss: 0.1868 - val_accuracy: 0.9293 - val_loss: 0.1568
Epoch 4/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9330 - loss: 0.1625 - val_accuracy: 0.9384 - val_loss: 0.1387
Epoch 5/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9394 - loss: 0.1443 - val_accuracy: 0.9459 - val_loss: 0.1261
Epoch 6/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9444 - loss: 0.1360 - val_accuracy: 0.9517 - val_loss: 0.1164
Epoch 7/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9465 - loss: 0.1246 - val_accuracy: 0.9520 - val_loss: 0.1105
Epoch 8/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step 

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-LSTM-Predict: Interactive LSTM Prediction

import numpy as np

# Assumes `scaler_lstm` and trained `model` are available

# 1. Prompt user for input values
age               = float(input("Enter Age (years): "))
height            = float(input("Enter Height (m): "))
heart_rate        = float(input("Enter Heart Rate (bpm): "))
blood_pressure    = float(input("Enter Blood Pressure (mmHg): "))
oxygen_saturation = float(input("Enter Oxygen Saturation (%): "))
activity_level    = float(input("Enter Activity Level (0=Low, 5=Medium, 10=High): "))
stress_level      = float(input("Enter Stress Level (0=Low, 5=Medium, 10=High): "))
bmi               = float(input("Enter BMI: "))

# 2. Create feature array
X_new = np.array([[age, height, heart_rate, blood_pressure,
                   oxygen_saturation, activity_level, stress_level, bmi]])

# 3. Scale and reshape for LSTM
X_new_scaled = scaler_lstm.transform(X_new)
X_new_lstm   = X_new_scaled.reshape((1, 1, X_new_scaled.shape[1]))

# 4. Predict with LSTM
y_proba = model.predict(X_new_lstm).flatten()[0]
y_pred  = int(y_proba >= 0.5)

# 5. Display results
risk_text = "HIGH RISK" if y_pred == 1 else "LOW RISK"
print(f"\nLSTM Prediction: {risk_text}")
#print(f"Predicted Probability of High Risk: {y_proba:.3f}")


Enter Age (years): 23
Enter Height (cm): 1.8
Enter Heart Rate (bpm): 88
Enter Blood Pressure (mmHg): 110
Enter Oxygen Saturation (%): 96
Enter Activity Level (0=Low, 5=Medium, 10=High): 10
Enter Stress Level (0=Low, 5=Medium, 10=High): 0
Enter BMI: 23.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

LSTM Prediction: LOW RISK
Predicted Probability of High Risk: nan


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-GRU: Train & Evaluate GRU Neural Network

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# 1. Prepare feature matrix X and target vector y (assumes df is cleaned and defined)
df['ActivityLevel_num'] = df['ActivityLevel'].map({'Low': 0, 'Medium': 1, 'High': 2})
X = df[['Age','Height','HeartRate','BloodPressure',
        'OxygenSaturation','ActivityLevel_num','StressLevel','BMI']].values # Use the new numeric column
y = df['HealthRisk'].astype(int).values

# 2. Split into train/test (70/30 split, stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 3. Feature Scaling
scaler_gru = StandardScaler()
X_train_scaled = scaler_gru.fit_transform(X_train)
X_test_scaled  = scaler_gru.transform(X_test)

# 3a. Handle any NaNs or Infs after scaling
X_train_scaled = np.nan_to_num(X_train_scaled, nan=0.0, posinf=0.0, neginf=0.0)
X_test_scaled  = np.nan_to_num(X_test_scaled,  nan=0.0, posinf=0.0, neginf=0.0)

# 4. Reshape for GRU: [samples, timesteps=1, features]
X_train_gru = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_gru  = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# 5. Build GRU Model
model_gru = Sequential([
    Input(shape=(1, X_train_gru.shape[2])),
    GRU(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 6. Train the model
history_gru = model_gru.fit(
    X_train_gru, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test_gru, y_test),
    verbose=1
)

# 7. Evaluate performance
y_proba_gru = model_gru.predict(X_test_gru).flatten()
y_pred_gru  = (y_proba_gru >= 0.5).astype(int)

print("=== GRU Performance ===")
print("Accuracy:", round(accuracy_score(y_test, y_pred_gru), 3))
if not np.isnan(y_proba_gru).any():
    print("ROC AUC:", round(roc_auc_score(y_test, y_proba_gru), 3))
else:
    print("ROC AUC: skipped (NaNs in predictions)")
print("\nClassification Report:\n", classification_report(y_test, y_pred_gru))


Epoch 1/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7451 - loss: 0.5209 - val_accuracy: 0.9104 - val_loss: 0.2225
Epoch 2/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9039 - loss: 0.2336 - val_accuracy: 0.9144 - val_loss: 0.1981
Epoch 3/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9106 - loss: 0.2055 - val_accuracy: 0.9192 - val_loss: 0.1814
Epoch 4/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9259 - loss: 0.1849 - val_accuracy: 0.9272 - val_loss: 0.1626
Epoch 5/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9292 - loss: 0.1731 - val_accuracy: 0.9341 - val_loss: 0.1463
Epoch 6/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9349 - loss: 0.1507 - val_accuracy: 0.9435 - val_loss: 0.1274
Epoch 7/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9426 - loss: 0.1360 - val_accuracy: 0.9515 - val_loss: 0.1161
Epoch 8/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9398 - loss: 0.1367 - val_accuracy: 0.

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-GRU-Predict: Interactive GRU Prediction

# Assumes `scaler_gru` and `model_gru` are already defined in the notebook

import numpy as np

# 1. Prompt user for input values
age               = float(input("Enter Age (years): "))
height            = float(input("Enter Height (cm): "))
heart_rate        = float(input("Enter Heart Rate (bpm): "))
blood_pressure    = float(input("Enter Blood Pressure (mmHg): "))
oxygen_saturation = float(input("Enter Oxygen Saturation (%): "))
activity_level    = float(input("Enter Activity Level (0=Low, 5=Medium, 10=High): "))
stress_level      = float(input("Enter Stress Level (0=Low, 5=Medium, 10=High): "))
bmi               = float(input("Enter BMI: "))

# 2. Create feature array and scale
X_new = np.array([[age, height, heart_rate, blood_pressure,
                   oxygen_saturation, activity_level, stress_level, bmi]])
X_new_scaled = scaler_gru.transform(X_new)

# 3. Reshape for GRU: [samples, timesteps=1, features]
X_new_gru = X_new_scaled.reshape((1, 1, X_new_scaled.shape[1]))

# 4. Predict with GRU
y_proba_gru = model_gru.predict(X_new_gru)[0, 0]
y_pred_gru  = int(y_proba_gru >= 0.5)

# 5. Display the results
risk_text = "HIGH RISK" if y_pred_gru == 1 else "LOW RISK"
print(f"\nGRU Prediction: {risk_text}")
print(f"Predicted Probability of High Risk: {y_proba_gru:.3f}")


Enter Age (years): 23
Enter Height (cm): 1.8
Enter Heart Rate (bpm): 88
Enter Blood Pressure (mmHg): 110
Enter Oxygen Saturation (%): 96
Enter Activity Level (0=Low, 5=Medium, 10=High): 10
Enter Stress Level (0=Low, 5=Medium, 10=High): 0
Enter BMI: 23.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

GRU Prediction: LOW RISK
Predicted Probability of High Risk: 0.032


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-RNN: Train & Evaluate Simple RNN Neural Network

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# 1. Prepare feature matrix X and target vector y (assumes df is cleaned and defined)
df['ActivityLevel_num'] = df['ActivityLevel'].map({'Low': 0, 'Medium': 1, 'High': 2})
X = df[['Age','Height','HeartRate','BloodPressure',
        'OxygenSaturation','ActivityLevel_num','StressLevel','BMI']].values # Use the new numeric column
y = df['HealthRisk'].astype(int).values

# 2. Split into train/test (70/30 split, stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 3. Feature Scaling
scaler_rnn = StandardScaler()
X_train_scaled = scaler_rnn.fit_transform(X_train)
X_test_scaled  = scaler_rnn.transform(X_test)

# 3a. Handle any NaNs or Infs after scaling
X_train_scaled = np.nan_to_num(X_train_scaled, nan=0.0, posinf=0.0, neginf=0.0)
X_test_scaled  = np.nan_to_num(X_test_scaled,  nan=0.0, posinf=0.0, neginf=0.0)

# 4. Reshape for RNN: [samples, timesteps=1, features]
X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_rnn  = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# 5. Build Simple RNN Model
model_rnn = Sequential([
    Input(shape=(1, X_train_rnn.shape[2])),
    SimpleRNN(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 6. Train the model
history_rnn = model_rnn.fit(
    X_train_rnn, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test_rnn, y_test),
    verbose=1
)

# 7. Evaluate performance
y_proba_rnn = model_rnn.predict(X_test_rnn).flatten()
y_pred_rnn  = (y_proba_rnn >= 0.5).astype(int)

print("=== Simple RNN Performance ===")
print("Accuracy:", round(accuracy_score(y_test, y_pred_rnn), 3))
if not np.isnan(y_proba_rnn).any():
    print("ROC AUC:", round(roc_auc_score(y_test, y_proba_rnn), 3))
else:
    print("ROC AUC: skipped (NaNs in predictions)")
print("\nClassification Report:\n", classification_report(y_test, y_pred_rnn))


Epoch 1/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8385 - loss: 0.4248 - val_accuracy: 0.9104 - val_loss: 0.2186
Epoch 2/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8952 - loss: 0.2434 - val_accuracy: 0.9181 - val_loss: 0.1896
Epoch 3/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9098 - loss: 0.2094 - val_accuracy: 0.9229 - val_loss: 0.1754
Epoch 4/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9141 - loss: 0.1886 - val_accuracy: 0.9283 - val_loss: 0.1614
Epoch 5/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9192 - loss: 0.1824 - val_accuracy: 0.9349 - val_loss: 0.1508
Epoch 6/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9247 - loss: 0.1716 - val_accuracy: 0.9395 - val_loss: 0.1416
Epoch 7/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9266 - loss: 0.1649 - val_accuracy: 0.9435 - val_loss: 0.1326
Epoch 8/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9347 - loss: 0.1550 - val_accuracy: 0.

## Section 3 Overview
This section focuses on integrating FIS outputs into ML:
- Extend dataset with fuzzy scores
- Train ML models on extended dataset
- Compare regression and classification metrics


#Section 3

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell Section3-1 (Revised): Generate Fuzzy Risk Features and Create New Dataset

import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy.control import Antecedent, Consequent, Rule, ControlSystem, ControlSystemSimulation

# Assumes `df` is the cleaned DataFrame with numeric columns:
# ['Age','Height','HeartRate','BloodPressure','OxygenSaturation',
#  'ActivityLevel','StressLevel','BMI','HealthRisk']

# 1. Redefine fuzzy variables
hr  = Antecedent(np.arange(40, 181, 1), 'heart_rate')
bp  = Antecedent(np.arange(80, 201, 1), 'blood_pressure')
ox  = Antecedent(np.arange(50, 101, 1), 'oxygen_saturation')
act = Antecedent(np.arange(0, 11, 1),   'activity_level')
st  = Antecedent(np.arange(0, 11, 1),   'stress_level')
ag  = Antecedent(np.arange(0, 101, 1),  'age')
ht  = Antecedent(np.arange(100, 221, 1),'height')
bm  = Antecedent(np.arange(10, 51, 1),  'bmi')
risk= Consequent(np.arange(0, 101, 1),   'health_risk')

# 2. Membership functions
hr['low'],    hr['normal'],    hr['high']     = fuzz.trapmf(hr.universe, [40,40,60,75]), fuzz.trimf(hr.universe, [60,75,100]), fuzz.trapmf(hr.universe, [90,110,180,180])
bp['low'],    bp['normal'],    bp['high']     = fuzz.trapmf(bp.universe, [80,80,90,110]), fuzz.trimf(bp.universe, [90,120,140]), fuzz.trapmf(bp.universe, [130,150,200,200])
ox['low'],    ox['normal']                      = fuzz.trapmf(ox.universe, [50,50,85,90]), fuzz.trimf(ox.universe, [85,95,100])
act['low'],   act['moderate'], act['high']     = fuzz.trimf(act.universe, [0,0,4]), fuzz.trimf(act.universe, [3,5,7]), fuzz.trimf(act.universe, [6,10,10])
st['low'],    st['medium'],    st['high']      = fuzz.trimf(st.universe, [0,0,3]), fuzz.trimf(st.universe, [2,5,8]), fuzz.trimf(st.universe, [7,10,10])
ag['young'],  ag['middle'],    ag['old']       = fuzz.trapmf(ag.universe, [0,0,20,30]), fuzz.trimf(ag.universe, [25,45,65]), fuzz.trapmf(ag.universe, [60,70,100,100])
ht['short'],  ht['avg'],       ht['tall']      = fuzz.trapmf(ht.universe, [100,100,140,160]), fuzz.trimf(ht.universe, [150,170,190]), fuzz.trapmf(ht.universe, [180,200,220,220])
bm['underweight'], bm['normal'], bm['overweight'], bm['obese'] = fuzz.trapmf(bm.universe, [10,10,15,18.5]), fuzz.trimf(bm.universe, [18.5,22,25]), fuzz.trimf(bm.universe, [24,27.5,30]), fuzz.trapmf(bm.universe, [29,32,50,50])
risk['low'], risk['high'] = fuzz.trimf(risk.universe, [0,0,50]), fuzz.trimf(risk.universe, [50,100,100])

# 3. Define 24 fuzzy rules
rules = [
    Rule(hr['high'] & bp['high'],       risk['high']),
    Rule(hr['high'] & ox['low'],        risk['high']),
    Rule(hr['high'] & st['high'],       risk['high']),
    Rule(bp['high'] & ox['low'],        risk['high']),
    Rule(bp['high'] & st['high'],       risk['high']),
    Rule(ox['low'] & st['high'],        risk['high']),
    Rule(st['high'] & bm['obese'],      risk['high']),
    Rule(ag['old'] & bp['high'],        risk['high']),
    Rule(ag['old'] & hr['high'],        risk['high']),
    Rule(ag['old'] & st['high'],        risk['high']),
    Rule(act['low'] & bm['obese'],      risk['high']),
    Rule(bm['obese'] & st['medium'],    risk['high']),

    Rule(hr['normal'] & bp['normal'] & ox['normal'], risk['low']),
    Rule(hr['normal'] & act['high'],     risk['low']),
    Rule(bp['normal'] & act['high'],     risk['low']),
    Rule(ox['normal'] & act['moderate'], risk['low']),
    Rule(st['low'] & act['high'],        risk['low']),
    Rule(bm['normal'] & ag['young'],     risk['low']),
    Rule(ag['young'] & ht['tall'],       risk['low']),
    Rule(bm['normal'] & ox['normal'],    risk['low']),
    Rule(bp['normal'] & st['low'],       risk['low']),
    Rule(hr['low'] & ox['normal'],       risk['low']),
    Rule(act['moderate'] & ht['avg'],    risk['low']),
    Rule(ag['middle'] & hr['normal'],    risk['low'])
]

# 4. Build control system
system = ControlSystem(rules)

# 5. Compute fuzzy features
fuzzy_scores, fuzzy_preds = [], []

for _, row in df.iterrows():
    sim = ControlSystemSimulation(system)
    sim.input['heart_rate']        = row['HeartRate']
    sim.input['blood_pressure']    = row['BloodPressure']
    sim.input['oxygen_saturation'] = row['OxygenSaturation']
    sim.input['activity_level']    = row['ActivityLevel']  # Ensure this is numeric (0, 1, or 2)
    sim.input['stress_level']      = row['StressLevel']
    sim.input['age']               = row['Age']
    sim.input['height']            = row['Height']
    sim.input['bmi']               = row['BMI']
    sim.compute()

    # Check if sim.output is empty before accessing values
    if sim.output:
        score = round(list(sim.output.values())[0], 1)
    else:
        # Handle the case when no rules are activated (e.g., assign a default value)
        score = 0  # or any other suitable default

    fuzzy_scores.append(score)
    fuzzy_preds.append(int(score >= 50))

# 6. Create new dataset with fuzzy features
df_new = df.copy()
df_new['fuzzy_score'] = fuzzy_scores
df_new['fuzzy_pred']  = fuzzy_preds

# 7. Preview extended dataset
print(df_new[['Age','HeartRate','fuzzy_score','fuzzy_pred','HealthRisk']].head())


    Age  HeartRate  fuzzy_score  fuzzy_pred  HealthRisk
0  75.0       57.0         58.8           1         1.0
1  53.0       83.0         17.4           0         1.0
2  49.0      138.0         83.3           1         1.0
3  40.0      139.0         83.3           1         1.0
4  57.0       67.0         19.1           0         1.0


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell Section3-2: Save Extended Dataset to CSV

# Specify the output CSV filename
output_file = '/content/sample_data/extended_health_risk_dataset.csv'

# Save df_new (with fuzzy_score and fuzzy_pred) to CSV
df_new.to_csv(output_file, index=False)

# Confirm save
print(f"Extended dataset with fuzzy features saved to: {output_file}")


Extended dataset with fuzzy features saved to: /content/sample_data/extended_health_risk_dataset.csv


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
df_new.head()

,PatientID,Age,Height,HeartRate,BloodPressure,OxygenSaturation,StressLevel,ActivityLevel,BMI,HealthRisk,ActivityLevel_num,fuzzy_score,fuzzy_pred
0,1.0,75.0,1.7,57.0,150.0,98.0,6.0,Medium,21.8,1.0,1,58.8,1
1,2.0,53.0,1.6,83.0,151.0,95.0,5.0,High,21.2,1.0,2,17.4,0
2,3.0,49.0,1.6,138.0,163.0,92.0,3.0,Medium,28.6,1.0,1,83.3,1
3,4.0,40.0,1.6,139.0,157.0,92.0,1.0,High,25.8,1.0,2,83.3,1
4,5.0,57.0,1.9,67.0,134.5,97.0,1.0,Low,25.1,1.0,0,19.1,0


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, roc_auc_score, classification_report
)

# Cell: Load extended dataset
df_ext = pd.read_csv('/content/sample_data/extended_health_risk_dataset.csv')

# Map ActivityLevel to numeric if still categorical
if df_ext['ActivityLevel'].dtype == object:
    df_ext['ActivityLevel'] = df_ext['ActivityLevel'].map({'Low': 0, 'Medium': 5, 'High': 10})

# Define feature matrix and targets
features = ['Age', 'Height', 'HeartRate', 'BloodPressure',
            'OxygenSaturation', 'ActivityLevel', 'StressLevel', 'BMI']
X = df_ext[features]
y_reg = df_ext['fuzzy_score']
y_clf = df_ext['HealthRisk'].astype(int)

# Split into train/test (70/30 stratified by classification target)
X_train, X_test, y_reg_train, y_reg_test, y_clf_train, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.3, random_state=42, stratify=y_clf
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Random Forest Regressor for fuzzy_score
rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(X_train_scaled, y_reg_train)
y_reg_pred = rfr.predict(X_test_scaled)

# Random Forest Classifier for HealthRisk
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train_scaled, y_clf_train)
y_clf_pred  = rfc.predict(X_test_scaled)
y_clf_proba = rfc.predict_proba(X_test_scaled)[:, 1]

# Evaluate regression performance
mse = mean_squared_error(y_reg_test, y_reg_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_reg_test, y_reg_pred)
r2 = r2_score(y_reg_test, y_reg_pred)

print("=== Random Forest Regression (Fuzzy Score) ===")
print(f"RMSE: {rmse:.3f}")
print(f"MAE:  {mae:.3f}")
print(f"R2:   {r2:.3f}\n")

# Evaluate classification performance
acc = accuracy_score(y_clf_test, y_clf_pred)
roc = roc_auc_score(y_clf_test, y_clf_proba)

print("=== Random Forest Classification (HealthRisk) ===")
print(f"Accuracy: {acc:.3f}")
print(f"ROC AUC:  {roc:.3f}\n")
print("Classification Report:")
print(classification_report(y_clf_test, y_clf_pred))


=== Random Forest Regression (Fuzzy Score) ===
RMSE: 3.405
MAE:  1.431
R2:   0.980

=== Random Forest Classification (HealthRisk) ===
Accuracy: 0.983
ROC AUC:  0.997

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       532
           1       0.99      0.99      0.99      3218

    accuracy                           0.98      3750
   macro avg       0.96      0.97      0.97      3750
weighted avg       0.98      0.98      0.98      3750



### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-Predict: Predict on New Patient Sample with Random Forest Models

import pandas as pd

# Define feature list (must match training)
features = ['Age', 'Height', 'HeartRate', 'BloodPressure',
            'OxygenSaturation', 'ActivityLevel', 'StressLevel', 'BMI']

# Example new patient data (replace with your values)
new_patient = {
    'Age': 60,
    'Height': 165,
    'HeartRate': 85,
    'BloodPressure': 140,
    'OxygenSaturation': 96,
    'ActivityLevel': 6,
    'StressLevel': 7,
    'BMI': 27.3
}

# Create DataFrame
X_new = pd.DataFrame([new_patient])

# Scale features using previously fitted scaler
X_new_scaled = scaler.transform(X_new[features])

# Predict fuzzy score (regression) and health risk (classification)
predicted_fuzzy_score = rfr.predict(X_new_scaled)[0]
predicted_health_risk = rfc.predict(X_new_scaled)[0]
predicted_risk_proba = rfc.predict_proba(X_new_scaled)[0, 1]

# Display results
print(f"Predicted Fuzzy Risk Score: {predicted_fuzzy_score:.1f}")
print(f"Predicted Health Risk (0=Low, 1=High): {predicted_health_risk}")
print(f"Probability of High Risk: {predicted_risk_proba:.3f}")


Predicted Fuzzy Risk Score: 47.3
Predicted Health Risk (0=Low, 1=High): 0
Probability of High Risk: 0.230


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-XGB: Train and Evaluate XGBoost Models

import numpy as np
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, roc_auc_score, classification_report
)

# 1. XGBoost Regressor for fuzzy_score
xgr = XGBRegressor(
    n_estimators=100,
    random_state=42,
    objective='reg:squarederror',
    eval_metric='rmse'
)
xgr.fit(X_train_scaled, y_reg_train)

# Predict regression target
y_reg_pred_xgb = xgr.predict(X_test_scaled)

# Evaluate regression performance
rmse_xgb = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_xgb))
mae_xgb  = mean_absolute_error(y_reg_test, y_reg_pred_xgb)
r2_xgb   = r2_score(y_reg_test, y_reg_pred_xgb)

print("=== XGBoost Regression (Fuzzy Score) ===")
print(f"RMSE: {rmse_xgb:.3f}")
print(f"MAE:  {mae_xgb:.3f}")
print(f"R2:   {r2_xgb:.3f}\n")

# 2. XGBoost Classifier for HealthRisk
xgbc = XGBClassifier(
    n_estimators=100,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgbc.fit(X_train_scaled, y_clf_train)

# Predict classification target
y_clf_pred_xgb  = xgbc.predict(X_test_scaled)
y_clf_proba_xgb = xgbc.predict_proba(X_test_scaled)[:, 1]

# Evaluate classification performance
acc_xgb = accuracy_score(y_clf_test, y_clf_pred_xgb)
roc_xgb = roc_auc_score(y_clf_test, y_clf_proba_xgb)

print("=== XGBoost Classification (HealthRisk) ===")
print(f"Accuracy: {acc_xgb:.3f}")
print(f"ROC AUC:  {roc_xgb:.3f}\n")
print("Classification Report:")
print(classification_report(y_clf_test, y_clf_pred_xgb))


=== XGBoost Regression (Fuzzy Score) ===
RMSE: 4.814
MAE:  3.002
R2:   0.961

=== XGBoost Classification (HealthRisk) ===
Accuracy: 0.982
ROC AUC:  0.996

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.94       532
           1       0.99      0.99      0.99      3218

    accuracy                           0.98      3750
   macro avg       0.96      0.97      0.96      3750
weighted avg       0.98      0.98      0.98      3750



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:56:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-XGB-Predict: Predict with XGBoost Models

import pandas as pd

# Define feature list (same as training)
features = ['Age', 'Height', 'HeartRate', 'BloodPressure',
            'OxygenSaturation', 'ActivityLevel', 'StressLevel', 'BMI']

# Example new patient data (replace with your values)
new_patient = {
    'Age': 60,
    'Height': 165,
    'HeartRate': 85,
    'BloodPressure': 140,
    'OxygenSaturation': 96,
    'ActivityLevel': 6,
    'StressLevel': 7,
    'BMI': 27.3
}

# Create DataFrame
X_new = pd.DataFrame([new_patient])

# Scale features using previously fitted scaler
X_new_scaled = scaler.transform(X_new[features])

# Predict fuzzy score using XGBRegressor
pred_fuzzy_xgb = xgr.predict(X_new_scaled)[0]

# Predict health risk using XGBClassifier
pred_health_xgb = xgbc.predict(X_new_scaled)[0]
pred_proba_xgb = xgbc.predict_proba(X_new_scaled)[0, 1]

# Display results
print(f"XGBoost Predicted Fuzzy Risk Score: {pred_fuzzy_xgb:.1f}")
print(f"XGBoost Predicted Health Risk (0=Low, 1=High): {pred_health_xgb}")
print(f"XGBoost Probability of High Risk: {pred_proba_xgb:.3f}")


XGBoost Predicted Fuzzy Risk Score: 45.1
XGBoost Predicted Health Risk (0=Low, 1=High): 0
XGBoost Probability of High Risk: 0.009


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-DT: Train and Evaluate Decision Tree on Extended Dataset

import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, roc_auc_score, classification_report
)

# 1. Load extended dataset
#df_ext = pd.read_csv('/co/data/extended_health_risk_dataset.csv')

# 2. Ensure ActivityLevel is numeric
if df_ext['ActivityLevel'].dtype == object:
    df_ext['ActivityLevel'] = df_ext['ActivityLevel'].map({'Low': 0, 'Medium': 5, 'High': 10})

# 3. Define features and targets
features = ['Age', 'Height', 'HeartRate', 'BloodPressure',
            'OxygenSaturation', 'ActivityLevel', 'StressLevel', 'BMI']
X = df_ext[features]
y_reg = df_ext['fuzzy_score']
y_clf = df_ext['HealthRisk'].astype(int)

# 4. Split into train/test (70/30 stratified)
X_train, X_test, y_reg_train, y_reg_test, y_clf_train, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.3, random_state=42, stratify=y_clf
)

# 5. Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 6. Train Decision Tree Regressor for fuzzy_score
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train_scaled, y_reg_train)
y_reg_pred_dt = dt_reg.predict(X_test_scaled)

# 7. Evaluate regression performance
rmse_dt = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_dt))
mae_dt  = mean_absolute_error(y_reg_test, y_reg_pred_dt)
r2_dt   = r2_score(y_reg_test, y_reg_pred_dt)

print("=== Decision Tree Regression (Fuzzy Score) ===")
print(f"RMSE: {rmse_dt:.3f}")
print(f"MAE:  {mae_dt:.3f}")
print(f"R2:   {r2_dt:.3f}\n")

# 8. Train Decision Tree Classifier for HealthRisk
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train_scaled, y_clf_train)
y_clf_pred_dt  = dt_clf.predict(X_test_scaled)
y_clf_proba_dt = dt_clf.predict_proba(X_test_scaled)[:, 1]

# 9. Evaluate classification performance
acc_dt = accuracy_score(y_clf_test, y_clf_pred_dt)
roc_dt = roc_auc_score(y_clf_test, y_clf_proba_dt)

print("=== Decision Tree Classification (HealthRisk) ===")
print(f"Accuracy: {acc_dt:.3f}")
print(f"ROC AUC:  {roc_dt:.3f}\n")
print("Classification Report:")
print(classification_report(y_clf_test, y_clf_pred_dt))


=== Decision Tree Regression (Fuzzy Score) ===
RMSE: 5.197
MAE:  1.869
R2:   0.954

=== Decision Tree Classification (HealthRisk) ===
Accuracy: 0.978
ROC AUC:  0.951

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       532
           1       0.99      0.99      0.99      3218

    accuracy                           0.98      3750
   macro avg       0.96      0.95      0.95      3750
weighted avg       0.98      0.98      0.98      3750



### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-DT-Predict: Predict with Decision Tree Models

import pandas as pd

# Define feature list (must match training)
features = ['Age', 'Height', 'HeartRate', 'BloodPressure',
            'OxygenSaturation', 'ActivityLevel', 'StressLevel', 'BMI']

# Example new patient data -- replace values as needed
new_patient = {
    'Age': 60,
    'Height': 165,
    'HeartRate': 85,
    'BloodPressure': 140,
    'OxygenSaturation': 96,
    'ActivityLevel': 6,
    'StressLevel': 7,
    'BMI': 27.3
}

# Create DataFrame for the new patient
X_new = pd.DataFrame([new_patient])

# Scale features using the previously fitted scaler
X_new_scaled = scaler.transform(X_new[features])

# Predict fuzzy risk score using Decision Tree Regressor
predicted_fuzzy_dt = dt_reg.predict(X_new_scaled)[0]

# Predict health risk and probability using Decision Tree Classifier
predicted_health_dt = dt_clf.predict(X_new_scaled)[0]
predicted_proba_dt = dt_clf.predict_proba(X_new_scaled)[0, 1]

# Display the results
print(f"Decision Tree Predicted Fuzzy Risk Score: {predicted_fuzzy_dt:.1f}")
print(f"Decision Tree Predicted Health Risk (0=Low, 1=High): {predicted_health_dt}")
print(f"Decision Tree Probability of High Risk: {predicted_proba_dt:.3f}")


Decision Tree Predicted Fuzzy Risk Score: 41.1
Decision Tree Predicted Health Risk (0=Low, 1=High): 0
Decision Tree Probability of High Risk: 0.000


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-LSTM: Train and Evaluate LSTM Models on Extended Dataset

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, roc_auc_score, classification_report
)

# 1. Load extended dataset
#df_ext = pd.read_csv('/mnt/data/extended_health_risk_dataset.csv')

# 2. Ensure ActivityLevel is numeric
if df_ext['ActivityLevel'].dtype == object:
    df_ext['ActivityLevel'] = df_ext['ActivityLevel'].map({'Low': 0, 'Medium': 5, 'High': 10})

# 3. Define features and targets
features = ['Age', 'Height', 'HeartRate', 'BloodPressure',
            'OxygenSaturation', 'ActivityLevel_num', 'StressLevel', 'BMI']
X = df_ext[features].values
y_reg = df_ext['fuzzy_score'].values
y_clf = df_ext['HealthRisk'].astype(int).values

# 4. Split into train/test (70/30 stratified on classification target)
X_train, X_test, y_reg_train, y_reg_test, y_clf_train, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.3, random_state=42, stratify=y_clf
)

# 5. Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 6. Reshape for LSTM: (samples, timesteps=8, features=1)
timesteps = X_train_scaled.shape[1]
X_train_seq = X_train_scaled.reshape(-1, timesteps, 1)
X_test_seq  = X_test_scaled.reshape(-1, timesteps, 1)

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# 7. Build and train LSTM Regressor for fuzzy_score
lstm_reg = Sequential([
    LSTM(32, input_shape=(timesteps, 1)),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])
lstm_reg.compile(optimizer='adam', loss='mse')

es_reg = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lstm_reg.fit(X_train_seq, y_reg_train, validation_split=0.2,
             epochs=200, batch_size=32, callbacks=[es_reg], verbose=2)

# 8. Predict and evaluate regression
y_reg_pred = lstm_reg.predict(X_test_seq).ravel()
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
mae  = mean_absolute_error(y_reg_test, y_reg_pred)
r2   = r2_score(y_reg_test, y_reg_pred)

print("=== LSTM Regression (Fuzzy Score) ===")
print(f"RMSE: {rmse:.3f}")
print(f"MAE:  {mae:.3f}")
print(f"R2:   {r2:.3f}\n")

Epoch 1/200
219/219 - 6s - 28ms/step - loss: 2768.4358 - val_loss: 1173.3362
Epoch 2/200
219/219 - 2s - 9ms/step - loss: 723.5148 - val_loss: 592.7803
Epoch 3/200
219/219 - 1s - 6ms/step - loss: 587.9599 - val_loss: 590.3287
Epoch 4/200
219/219 - 1s - 6ms/step - loss: 544.3098 - val_loss: 497.4054
Epoch 5/200
219/219 - 1s - 6ms/step - loss: 463.0480 - val_loss: 469.7641
Epoch 6/200
219/219 - 2s - 7ms/step - loss: 435.0078 - val_loss: 438.3211
Epoch 7/200
219/219 - 2s - 11ms/step - loss: 415.2349 - val_loss: 413.9706
Epoch 8/200
219/219 - 1s - 5ms/step - loss: 389.4698 - val_loss: 369.8389
Epoch 9/200
219/219 - 1s - 5ms/step - loss: 326.0557 - val_loss: 302.2151
Epoch 10/200
219/219 - 1s - 6ms/step - loss: 262.5735 - val_loss: 262.1398
Epoch 11/200
219/219 - 1s - 6ms/step - loss: 242.2946 - val_loss: 246.1878
Epoch 12/200
219/219 - 2s - 8ms/step - loss: 230.3144 - val_loss: 234.3859
Epoch 13/200
219/219 - 2s - 10ms/step - loss: 219.8958 - val_loss: 224.1486
Epoch 14/200
219/219 - 3s - 1

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# 9. Build and train LSTM Classifier for HealthRisk
lstm_clf = Sequential([
    LSTM(32, input_shape=(timesteps, 1)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
lstm_clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

es_clf = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lstm_clf.fit(X_train_seq, y_clf_train, validation_split=0.2,
             epochs=50, batch_size=32, callbacks=[es_clf], verbose=2)

# 10. Predict and evaluate classification
y_clf_proba = lstm_clf.predict(X_test_seq).ravel()
y_clf_pred  = (y_clf_proba >= 0.5).astype(int)

acc = accuracy_score(y_clf_test, y_clf_pred)
roc = roc_auc_score(y_clf_test, y_clf_proba)

print("=== LSTM Classification (HealthRisk) ===")
print(f"Accuracy: {acc:.3f}")
print(f"ROC AUC:  {roc:.3f}\n")
print("Classification Report:")
print(classification_report(y_clf_test, y_clf_pred))

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 - 4s - 17ms/step - accuracy: 0.8567 - loss: 0.4270 - val_accuracy: 0.8611 - val_loss: 0.3900
Epoch 2/50
219/219 - 1s - 5ms/step - accuracy: 0.8576 - loss: 0.3808 - val_accuracy: 0.8611 - val_loss: 0.3657
Epoch 3/50
219/219 - 1s - 5ms/step - accuracy: 0.8576 - loss: 0.3614 - val_accuracy: 0.8611 - val_loss: 0.3380
Epoch 4/50
219/219 - 1s - 6ms/step - accuracy: 0.8600 - loss: 0.3192 - val_accuracy: 0.8754 - val_loss: 0.2834
Epoch 5/50
219/219 - 2s - 7ms/step - accuracy: 0.8781 - loss: 0.2782 - val_accuracy: 0.9046 - val_loss: 0.2347
Epoch 6/50
219/219 - 2s - 11ms/step - accuracy: 0.9136 - loss: 0.2053 - val_accuracy: 0.9400 - val_loss: 0.1536
Epoch 7/50
219/219 - 1s - 5ms/step - accuracy: 0.9356 - loss: 0.1498 - val_accuracy: 0.9520 - val_loss: 0.1345
Epoch 8/50
219/219 - 1s - 5ms/step - accuracy: 0.9440 - loss: 0.1288 - val_accuracy: 0.9497 - val_loss: 0.1164
Epoch 9/50
219/219 - 1s - 6ms/step - accuracy: 0.9480 - loss: 0.1228 - val_accuracy: 0.9497 - val_loss: 0.1080
Epoch 10/5

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-LSTM-Predict: Predict with LSTM Models

import pandas as pd
import numpy as np

# Define feature list (must match training)
features = ['Age', 'Height', 'HeartRate', 'BloodPressure',
            'OxygenSaturation', 'ActivityLevel', 'StressLevel', 'BMI']

# Example new patient data (replace values as needed)
new_patient = {
    'Age': 23,
    'Height': 1.8,
    'HeartRate': 125,
    'BloodPressure': 110,
    'OxygenSaturation': 96,
    'ActivityLevel': 10,
    'StressLevel': 2,
    'BMI': 22.6
}

# Create DataFrame for the new patient
X_new = pd.DataFrame([new_patient])

# Scale features using previously fitted scaler
X_new_scaled = scaler.transform(X_new[features])

# Reshape for LSTM: (samples, timesteps, features=1)
timesteps = X_new_scaled.shape[1]
X_new_seq = X_new_scaled.reshape(-1, timesteps, 1)

# Predict fuzzy risk score using LSTM Regressor
predicted_fuzzy_lstm = lstm_reg.predict(X_new_seq).ravel()[0]

# Predict health risk probability using LSTM Classifier
pred_proba_lstm = lstm_clf.predict(X_new_seq).ravel()[0]
predicted_health_lstm = int(pred_proba_lstm >= 0.5)

# Display the results
print(f"LSTM Predicted Fuzzy Risk Score: {predicted_fuzzy_lstm:.1f}")
print(f"LSTM Predicted Health Risk (0=Low, 1=High): {predicted_health_lstm}")
print(f"LSTM Probability of High Risk: {pred_proba_lstm:.3f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
LSTM Predicted Fuzzy Risk Score: 20.7
LSTM Predicted Health Risk (0=Low, 1=High): 0
LSTM Probability of High Risk: 0.475


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# 7. Build and train GRU Regressor for fuzzy_score
gru_reg = Sequential([
    GRU(32, input_shape=(timesteps, 1)),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])
gru_reg.compile(optimizer='adam', loss='mse')

es_reg = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
gru_reg.fit(X_train_seq, y_reg_train, validation_split=0.2,
            epochs=50, batch_size=32, callbacks=[es_reg], verbose=2)

# 8. Predict and evaluate regression
y_reg_pred = gru_reg.predict(X_test_seq).ravel()
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
mae  = mean_absolute_error(y_reg_test, y_reg_pred)
r2   = r2_score(y_reg_test, y_reg_pred)

print("=== GRU Regression (Fuzzy Score) ===")
print(f"RMSE: {rmse:.3f}")
print(f"MAE:  {mae:.3f}")
print(f"R2:   {r2:.3f}\n")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 - 5s - 25ms/step - loss: 2737.2783 - val_loss: 1278.8661
Epoch 2/50
219/219 - 1s - 6ms/step - loss: 785.4252 - val_loss: 596.4667
Epoch 3/50
219/219 - 2s - 11ms/step - loss: 589.4017 - val_loss: 591.0455
Epoch 4/50
219/219 - 1s - 6ms/step - loss: 583.1786 - val_loss: 558.0397
Epoch 5/50
219/219 - 1s - 6ms/step - loss: 496.4974 - val_loss: 476.6867
Epoch 6/50
219/219 - 3s - 13ms/step - loss: 446.0516 - val_loss: 444.9928
Epoch 7/50
219/219 - 1s - 6ms/step - loss: 425.9038 - val_loss: 428.2554
Epoch 8/50
219/219 - 3s - 11ms/step - loss: 407.6209 - val_loss: 407.0692
Epoch 9/50
219/219 - 3s - 12ms/step - loss: 353.9898 - val_loss: 317.5879
Epoch 10/50
219/219 - 1s - 6ms/step - loss: 271.6027 - val_loss: 253.9063
Epoch 11/50
219/219 - 1s - 7ms/step - loss: 230.2050 - val_loss: 243.9386
Epoch 12/50
219/219 - 2s - 8ms/step - loss: 217.5844 - val_loss: 229.8808
Epoch 13/50
219/219 - 2s - 9ms/step - loss: 210.5770 - val_loss: 217.1478
Epoch 14/50
219/219 - 3s - 12ms/step - loss: 207.88

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# 9. Build and train GRU Classifier for HealthRisk
gru_clf = Sequential([
    GRU(32, input_shape=(timesteps, 1)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
gru_clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

es_clf = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
gru_clf.fit(X_train_seq, y_clf_train, validation_split=0.2,
            epochs=50, batch_size=32, callbacks=[es_clf], verbose=2)

# 10. Predict and evaluate classification
y_clf_proba = gru_clf.predict(X_test_seq).ravel()
y_clf_pred  = (y_clf_proba >= 0.5).astype(int)

acc = accuracy_score(y_clf_test, y_clf_pred)
roc = roc_auc_score(y_clf_test, y_clf_proba)

print("=== GRU Classification (HealthRisk) ===")
print(f"Accuracy: {acc:.3f}")
print(f"ROC AUC:  {roc:.3f}\n")
print("Classification Report:")
print(classification_report(y_clf_test, y_clf_pred))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
219/219 - 9s - 41ms/step - accuracy: 0.8427 - loss: 0.4364 - val_accuracy: 0.8611 - val_loss: 0.3775
Epoch 2/50
219/219 - 2s - 10ms/step - accuracy: 0.8576 - loss: 0.3460 - val_accuracy: 0.8720 - val_loss: 0.2918
Epoch 3/50
219/219 - 2s - 7ms/step - accuracy: 0.8827 - loss: 0.2736 - val_accuracy: 0.9069 - val_loss: 0.2328
Epoch 4/50
219/219 - 2s - 9ms/step - accuracy: 0.9154 - loss: 0.2141 - val_accuracy: 0.9303 - val_loss: 0.1758
Epoch 5/50
219/219 - 2s - 7ms/step - accuracy: 0.9373 - loss: 0.1599 - val_accuracy: 0.9503 - val_loss: 0.1272
Epoch 6/50
219/219 - 1s - 6ms/step - accuracy: 0.9487 - loss: 0.1277 - val_accuracy: 0.9577 - val_loss: 0.1117
Epoch 7/50
219/219 - 1s - 6ms/step - accuracy: 0.9526 - loss: 0.1148 - val_accuracy: 0.9583 - val_loss: 0.1005
Epoch 8/50
219/219 - 1s - 6ms/step - accuracy: 0.9563 - loss: 0.1075 - val_accuracy: 0.9623 - val_loss: 0.0992
Epoch 9/50
219/219 - 3s - 12ms/step - accuracy: 0.9616 - loss: 0.0999 - val_accuracy: 0.9623 - val_loss: 0.089

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-GRU-Predict: Predict with GRU Models

import pandas as pd

# Define feature list (must match training)
features = ['Age', 'Height', 'HeartRate', 'BloodPressure',
            'OxygenSaturation', 'ActivityLevel', 'StressLevel', 'BMI']

# Example new patient data (replace values as needed)
new_patient = {
    'Age': 60,
    'Height': 165,
    'HeartRate': 85,
    'BloodPressure': 140,
    'OxygenSaturation': 96,
    'ActivityLevel': 6,
    'StressLevel': 7,
    'BMI': 27.3
}

# Create DataFrame for the new patient
X_new = pd.DataFrame([new_patient])

# Scale features using the previously fitted scaler
X_new_scaled = scaler.transform(X_new[features])

# Reshape for GRU: (samples, timesteps, features=1)
timesteps = X_new_scaled.shape[1]
X_new_seq = X_new_scaled.reshape(-1, timesteps, 1)

# Predict fuzzy risk score using GRU Regressor
predicted_fuzzy_gru = gru_reg.predict(X_new_seq).ravel()[0]

# Predict health risk probability using GRU Classifier
pred_proba_gru = gru_clf.predict(X_new_seq).ravel()[0]
predicted_health_gru = int(pred_proba_gru >= 0.5)

# Display the results
print(f"GRU Predicted Fuzzy Risk Score: {predicted_fuzzy_gru:.1f}")
print(f"GRU Predicted Health Risk (0=Low, 1=High): {predicted_health_gru}")
print(f"GRU Probability of High Risk: {pred_proba_gru:.3f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
GRU Predicted Fuzzy Risk Score: 83.8
GRU Predicted Health Risk (0=Low, 1=High): 1
GRU Probability of High Risk: 0.952


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:

# 7. Build and train RNN Regressor for fuzzy_score
rnn_reg = Sequential([
    SimpleRNN(32, input_shape=(timesteps, 1)),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])
rnn_reg.compile(optimizer='adam', loss='mse')

es_reg = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
rnn_reg.fit(X_train_seq, y_reg_train, validation_split=0.2,
            epochs=50, batch_size=32, callbacks=[es_reg], verbose=2)

# 8. Predict and evaluate regression
y_reg_pred = rnn_reg.predict(X_test_seq).ravel()
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
mae  = mean_absolute_error(y_reg_test, y_reg_pred)
r2   = r2_score(y_reg_test, y_reg_pred)

print("=== RNN Regression (Fuzzy Score) ===")
print(f"RMSE: {rmse:.3f}")
print(f"MAE:  {mae:.3f}")
print(f"R2:   {r2:.3f}\n")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 - 4s - 17ms/step - loss: 2404.7588 - val_loss: 727.1211
Epoch 2/50
219/219 - 1s - 4ms/step - loss: 604.5419 - val_loss: 590.8762
Epoch 3/50
219/219 - 1s - 6ms/step - loss: 584.8762 - val_loss: 572.4402
Epoch 4/50
219/219 - 1s - 6ms/step - loss: 535.0455 - val_loss: 501.4815
Epoch 5/50
219/219 - 1s - 4ms/step - loss: 453.5869 - val_loss: 360.9736
Epoch 6/50
219/219 - 2s - 8ms/step - loss: 316.4787 - val_loss: 275.2428
Epoch 7/50
219/219 - 1s - 6ms/step - loss: 262.9080 - val_loss: 265.8533
Epoch 8/50
219/219 - 2s - 9ms/step - loss: 232.9850 - val_loss: 231.1668
Epoch 9/50
219/219 - 1s - 6ms/step - loss: 218.1692 - val_loss: 221.5294
Epoch 10/50
219/219 - 1s - 6ms/step - loss: 209.3161 - val_loss: 216.3671
Epoch 11/50
219/219 - 1s - 5ms/step - loss: 202.1777 - val_loss: 214.3323
Epoch 12/50
219/219 - 1s - 7ms/step - loss: 195.3956 - val_loss: 203.8511
Epoch 13/50
219/219 - 2s - 11ms/step - loss: 191.5789 - val_loss: 201.2696
Epoch 14/50
219/219 - 3s - 12ms/step - loss: 186.9962 -

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# 9. Build and train RNN Classifier for HealthRisk
rnn_clf = Sequential([
    SimpleRNN(32, input_shape=(timesteps, 1)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
rnn_clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

es_clf = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
rnn_clf.fit(X_train_seq, y_clf_train, validation_split=0.2,
            epochs=50, batch_size=32, callbacks=[es_clf], verbose=2)

# 10. Predict and evaluate classification
y_clf_proba = rnn_clf.predict(X_test_seq).ravel()
y_clf_pred  = (y_clf_proba >= 0.5).astype(int)

acc = accuracy_score(y_clf_test, y_clf_pred)
roc = roc_auc_score(y_clf_test, y_clf_proba)

print("=== RNN Classification (HealthRisk) ===")
print(f"Accuracy: {acc:.3f}")
print(f"ROC AUC:  {roc:.3f}\n")
print("Classification Report:")
print(classification_report(y_clf_test, y_clf_pred))

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 - 3s - 14ms/step - accuracy: 0.9003 - loss: 0.2695 - val_accuracy: 0.9206 - val_loss: 0.2004
Epoch 2/50
219/219 - 1s - 4ms/step - accuracy: 0.9274 - loss: 0.1824 - val_accuracy: 0.9434 - val_loss: 0.1576
Epoch 3/50
219/219 - 1s - 4ms/step - accuracy: 0.9427 - loss: 0.1485 - val_accuracy: 0.9469 - val_loss: 0.1334
Epoch 4/50
219/219 - 1s - 6ms/step - accuracy: 0.9519 - loss: 0.1247 - val_accuracy: 0.9509 - val_loss: 0.1166
Epoch 5/50
219/219 - 1s - 4ms/step - accuracy: 0.9563 - loss: 0.1140 - val_accuracy: 0.9577 - val_loss: 0.1133
Epoch 6/50
219/219 - 1s - 4ms/step - accuracy: 0.9596 - loss: 0.1049 - val_accuracy: 0.9640 - val_loss: 0.0972
Epoch 7/50
219/219 - 1s - 6ms/step - accuracy: 0.9627 - loss: 0.0963 - val_accuracy: 0.9600 - val_loss: 0.0962
Epoch 8/50
219/219 - 1s - 7ms/step - accuracy: 0.9614 - loss: 0.0939 - val_accuracy: 0.9629 - val_loss: 0.0872
Epoch 9/50
219/219 - 2s - 9ms/step - accuracy: 0.9659 - loss: 0.0876 - val_accuracy: 0.9663 - val_loss: 0.0857
Epoch 10/50

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell ML-RNN-Predict: Predict with RNN Models

import pandas as pd

# Define feature list (must match training)
features = ['Age', 'Height', 'HeartRate', 'BloodPressure',
            'OxygenSaturation', 'ActivityLevel', 'StressLevel', 'BMI']

# Example new patient data (replace values as needed)
new_patient = {
    'Age': 23,
    'Height': 1.8,
    'HeartRate': 85,
    'BloodPressure': 110,
    'OxygenSaturation': 96,
    'ActivityLevel': 2,
    'StressLevel': 1,
    'BMI': 27.3
}

# Create DataFrame for the new patient
X_new = pd.DataFrame([new_patient])

# Scale features using the previously fitted scaler
X_new_scaled = scaler.transform(X_new[features])

# Reshape for RNN: (samples, timesteps, features=1)
X_new_seq = X_new_scaled.reshape(-1, timesteps, 1)

# Predict fuzzy risk score using RNN Regressor
predicted_fuzzy_rnn = rnn_reg.predict(X_new_seq).ravel()[0]

# Predict health risk probability using RNN Classifier
pred_proba_rnn = rnn_clf.predict(X_new_seq).ravel()[0]
predicted_health_rnn = int(pred_proba_rnn >= 0.5)

# Display the results
print(f"RNN Predicted Fuzzy Risk Score: {predicted_fuzzy_rnn:.1f}")
print(f"RNN Predicted Health Risk (0=Low, 1=High): {predicted_health_rnn}")
print(f"RNN Probability of High Risk: {pred_proba_rnn:.3f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
RNN Predicted Fuzzy Risk Score: 14.8
RNN Predicted Health Risk (0=Low, 1=High): 0
RNN Probability of High Risk: 0.053


## Section 4 Overview
Here we refine the fuzzy rule base using ML insights:
- Extract feature importances
- Derive new fuzzy rules from top features
- Update and evaluate the refined FIS


#Section 4

### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

# Load dataset
df = pd.read_csv('/content/sample_data/extended_health_risk_dataset.csv')
if df['ActivityLevel'].dtype == object:
    df['ActivityLevel'] = df['ActivityLevel'].map({'Low': 0, 'Medium': 5, 'High': 10})

# Features & target
features = ['Age','Height','HeartRate','BloodPressure',
            'OxygenSaturation','ActivityLevel','StressLevel','BMI']
X = df[features]
y = df['HealthRisk'].astype(int)

# Split & scale
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Train models
rf  = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_train_scaled, y_train)
xgb = XGBClassifier(n_estimators=100, eval_metric='logloss', random_state=42).fit(X_train_scaled, y_train)
dt  = DecisionTreeClassifier(random_state=42).fit(X_train_scaled, y_train)

# Build insights DataFrame
insights = pd.DataFrame({
    'feature': features,
    'rf_importance':   rf.feature_importances_,
    'xgb_importance':  xgb.feature_importances_,
    'dt_importance':   dt.feature_importances_
})

insights.to_csv('model_importances.csv', index=False)
print("Saved feature importances to model_importances.csv")


Saved feature importances to model_importances.csv


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Fuzzy Inference System with Refined Rule Base

import numpy as np
import skfuzzy as fuzz
from skfuzzy.control import (
    Antecedent, Consequent, Rule,
    ControlSystem, ControlSystemSimulation
)

# 1. Define Fuzzy Variables (Antecedents and Consequent)
hr   = Antecedent(np.arange(40, 181, 1), 'heart_rate')
bp   = Antecedent(np.arange(80, 201, 1), 'blood_pressure')
ox   = Antecedent(np.arange(50, 101, 1), 'oxygen_saturation')
act  = Antecedent(np.arange(0, 11,   1), 'activity_level')
st   = Antecedent(np.arange(0, 11,   1), 'stress_level')
ag   = Antecedent(np.arange(0, 101,  1), 'age')
ht   = Antecedent(np.arange(0, 5, 1), 'height')
bm   = Antecedent(np.arange(10, 51,  1), 'bmi')

risk = Consequent(np.arange(0, 101, 1), 'health_risk')

# 2. Membership Functions
hr['low']    = fuzz.trapmf(hr.universe, [40,  40,  60,  75])
hr['normal'] = fuzz.trimf(hr.universe, [60,  75, 100])
hr['high']   = fuzz.trapmf(hr.universe, [90, 110, 180, 180])

bp['low']    = fuzz.trapmf(bp.universe, [80,  80,  90,  110])
bp['normal'] = fuzz.trimf(bp.universe, [90, 120, 140])
bp['high']   = fuzz.trapmf(bp.universe, [130,150, 200, 200])

ox['low']    = fuzz.trapmf(ox.universe, [50,  50,  85,  90])
ox['normal'] = fuzz.trimf(ox.universe, [85,  95, 100])

act['low']      = fuzz.trimf(act.universe, [0, 0, 4])
act['moderate'] = fuzz.trimf(act.universe, [3, 5, 7])
act['high']     = fuzz.trimf(act.universe, [6,10,10])

st['low']    = fuzz.trimf(st.universe, [0, 0, 3])
st['medium'] = fuzz.trimf(st.universe, [2, 5, 8])
st['high']   = fuzz.trimf(st.universe, [7,10,10])

ag['young']  = fuzz.trapmf(ag.universe, [0,  0,  20, 30])
ag['middle'] = fuzz.trimf(ag.universe, [25, 45, 65])
ag['old']    = fuzz.trapmf(ag.universe, [60, 70,100,100])

ht['short'] = fuzz.trapmf(ht.universe, [100,100,140,160])
ht['avg']   = fuzz.trimf(ht.universe, [150,170,190])
ht['tall']  = fuzz.trapmf(ht.universe, [180,200,220,220])

bm['underweight'] = fuzz.trapmf(bm.universe, [10,  10,  15,  18.5])
bm['normal']      = fuzz.trimf(bm.universe, [18.5,22,  25])
bm['overweight']  = fuzz.trimf(bm.universe, [24, 27.5,30])
bm['obese']       = fuzz.trapmf(bm.universe, [29, 32,  50, 50])

risk['low']  = fuzz.trimf(risk.universe, [0,  0,  50])
risk['high'] = fuzz.trimf(risk.universe, [50,100,100])

# 3. Original 24 Fuzzy Rules
rules = [
    Rule(hr['high'] & bp['high'],                   risk['high']),
    Rule(hr['high'] & ox['low'],                    risk['high']),
    Rule(hr['high'] & st['high'],                   risk['high']),
    Rule(bp['high'] & ox['low'],                    risk['high']),
    Rule(bp['high'] & st['high'],                   risk['high']),
    Rule(ox['low']  & st['high'],                   risk['high']),
    Rule(st['high'] & bm['obese'],                  risk['high']),
    Rule(ag['old']  & bp['high'],                   risk['high']),
    Rule(ag['old']  & hr['high'],                   risk['high']),
    Rule(ag['old']  & st['high'],                   risk['high']),
    Rule(act['low'] & bm['obese'],                  risk['high']),
    Rule(bm['obese'] & st['medium'],                risk['high']),

    Rule(hr['normal'] & bp['normal'] & ox['normal'],risk['low']),
    Rule(hr['normal'] & act['high'],                risk['low']),
    Rule(bp['normal'] & act['high'],                risk['low']),
    Rule(ox['normal'] & act['moderate'],            risk['low']),
    Rule(st['low']    & act['high'],                risk['low']),
    Rule(bm['normal'] & ag['young'],                risk['low']),
    Rule(ag['young']  & ht['tall'],                 risk['low']),
    Rule(bm['normal'] & ox['normal'],               risk['low']),
    Rule(bp['normal'] & st['low'],                  risk['low']),
    Rule(hr['low']    & ox['normal'],               risk['low']),
    Rule(act['moderate'] & ht['avg'],               risk['low']),
    Rule(ag['middle']    & hr['normal'],            risk['low'])
]

# 4. Refined Rules Based on ML/DL Insights (Top Features: BP, BMI, HR, Age, O₂)
high_refinements = [
    Rule(bp['high']                      & bm['obese'],            risk['high']),
    Rule(bp['high'] & hr['high']         & st['high'],             risk['high']),
    Rule(hr['high'] & ag['old']          & bm['overweight'],       risk['high']),
    Rule(ox['low']  & bp['high']         & bm['obese'],            risk['high']),
    Rule(hr['high'] & ox['low']          & st['medium'],           risk['high']),
]

low_refinements = [
    Rule(act['high']  & st['low']  & ox['normal'], risk['low']),
    Rule(ag['young'] & bm['normal'] & bp['normal'], risk['low']),
    Rule(hr['normal'] & bp['normal'] & bm['normal'],risk['low']),
    Rule(ox['normal'] & act['moderate'] & st['medium'], risk['low']),
    Rule(act['high']  & ag['middle'] & st['low'],    risk['low']),
]

# 5. Combine and Build Control System
refined_rules = rules + high_refinements + low_refinements
health_ctrl   = ControlSystem(refined_rules)
health_sim    = ControlSystemSimulation(health_ctrl)

# 6. Prediction Function
def predict_binary_risk(
    heart_rate, blood_pressure, oxygen_saturation,
    activity_level, stress_level, age, height, bmi
):
    """Returns (risk_score [0–100], risk_binary [0/1])."""
    health_sim.input['heart_rate']        = heart_rate
    health_sim.input['blood_pressure']    = blood_pressure
    health_sim.input['oxygen_saturation'] = oxygen_saturation
    health_sim.input['activity_level']    = activity_level
    health_sim.input['stress_level']      = stress_level
    health_sim.input['age']               = age
    health_sim.input['height']            = height
    health_sim.input['bmi']               = bmi
    health_sim.compute()
    score  = round(health_sim.output['health_risk'], 1)
    binary = int(score >= 50)
    return score, binary

# Example
# score, pred = predict_binary_risk(80,120,95,7,4,50,170,24)
# print("Refined Score:", score, "Binary:", pred)


### Description: *Add a brief explanation of this cell's purpose and operations.*

In [ ]:
# Cell Prediction: Fuzzy Inference with Refined Rule Base

# Assumes the refined FIS and `predict_binary_risk` are already defined in the notebook.

# 1. Define a new patient record (replace with your own values)
new_patient = {
    'heart_rate':        88,    # bpm
    'blood_pressure':    110,   # mmHg
    'oxygen_saturation': 96,    # %
    'activity_level':    10,     # 0–10
    'stress_level':      0,     # 0–10
    'age':               23,    # years
    'height':            1.8,   # m
    'bmi':               22.6   # kg/m²
}

# 2. Unpack and predict
score, binary = predict_binary_risk(
    new_patient['heart_rate'],
    new_patient['blood_pressure'],
    new_patient['oxygen_saturation'],
    new_patient['activity_level'],
    new_patient['stress_level'],
    new_patient['age'],
    new_patient['height'],
    new_patient['bmi']
)

# 3. Display results
print(f"Predicted Fuzzy Risk Score: {score:.1f} / 100")
print("Predicted Risk Category:", "HIGH RISK" if binary else "LOW RISK")


Predicted Fuzzy Risk Score: 16.7 / 100
Predicted Risk Category: LOW RISK
